<a href="https://colab.research.google.com/github/humb3rt84/UT/blob/main/GEVAL_scalable_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install
import os

!pip install deepeval
!pip install nest_asyncio
!pip install -U deepeval

openapi_key_geval = os.environ.get("OPENAI_KEY_GEVAL")


In [8]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric

# Create a sample test case
test_case = LLMTestCase(
    input="Ask to my boss in Teams chat for a 3 weeks to write my doctorate thesis.",
    actual_output="Hi Sean! Quick question—would it be possible to take 3 weeks to focus on writing my doctorate thesis? I’ll make sure everything is transitioned smoothly before then. Let me know if we can chat about it! Thanks so much!",
    expected_output="Hi Sean! Would you please approve my 3 weeks vacation petitions so I can focus on writing my thesis. This is the final stage on my doctorate journey and I would like to use all my energy on it. Thanks in advance for your consideration.",
    retrieval_context=[
        "I am doing my Doctorate at Hult Business School while working at Microsoft as a Senior Product Manager"
    ]
)

# Initialize the metric and run the measurement
relevancy_metric = AnswerRelevancyMetric(threshold=0.5)
relevancy_metric.measure(test_case)

# Print out the numeric score
print("Relevancy score:", relevancy_metric.score)
print("Explanation:", relevancy_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Relevancy score: 0.6
Explanation: The score is 0.60 because the output includes a greeting and an expression of gratitude that do not address the request for time off to write a doctorate thesis. However, it partially addresses the main request, which is why it is not lower.


In [13]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

# 1. Create a sample test case.
test_case = LLMTestCase(
    input="Ask my boss in Teams chat for a 3-week period to write my doctorate thesis.",
    actual_output=(
        "Hi Sean! Quick question—would it be possible to take 3 weeks to focus "
        "on writing my doctorate thesis? I’ll make sure everything is transitioned smoothly "
        "before then. Let me know if we can chat about it! Thanks so much!"
    ),
    expected_output=(
        "Hi Sean! Would you please approve my 3 weeks vacation petitions so I can focus "
        "on writing my thesis. This is the final stage on my doctorate journey and I would like "
        "to use all my energy on it. Thanks in advance for your consideration."
    ),
    retrieval_context=[
        "I am doing my Doctorate at Hult Business School while working at Microsoft as a Senior Product Manager"
    ]
)

# 2. Define a G-Eval metric for 'Answer Relevancy'.
#    Either provide a single 'criteria' OR stepwise 'evaluation_steps'. Below is one way using 'criteria'.
relevancy_metric = GEval(
    name="AnswerRelevancy",
    criteria=(
        "You are to decide how relevant the actual output is to the user's request (input) "
        "and how well it matches the expected output. If the actual output closely addresses "
        "the input, references necessary details provided in the retrieval context, and does "
        "not omit critical information from the expected output, then it should receive a high score. "
        "Minor stylistic differences are acceptable and should not be penalized."
    ),
    # Which parameters are relevant for the evaluation
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.EXPECTED_OUTPUT,
        LLMTestCaseParams.RETRIEVAL_CONTEXT
    ],
    threshold=0.5,      # Optional: define the threshold for a 'passing' score
    model="gpt-4o",     # Optional: specify a GPT model or a custom model
    async_mode=True,    # Optional: run concurrent calls if you have multiple test cases
    verbose_mode=False  # Optional: print intermediate steps
)

# 3. Run the measurement
relevancy_metric.measure(test_case)

# 4. Print the numeric score and explanation
print("Relevancy Score:", relevancy_metric.score)
print("Explanation:", relevancy_metric.reason)




Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Relevancy Score: 0.5917270321253567
Explanation: The actual output addresses the input by requesting time off but is more formal and indirect than the expected output. It does not explicitly reference the retrieval context, such as working at Microsoft or attending Hult Business School, which could be relevant for context.
